In [1]:
import gym
import numpy as np
import homework.hw1.load_policy as load_policy
import homework.hw1.tf_util as tf_util
import tensorflow as tf

dim = 17

/Users/torbensell/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/torbensell/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/torbensell/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/torbensell/anaconda3/lib/python3.6/site-packages/tensorflow/

In [2]:
"""

NEEDED FUNCTIONS

"""
    
def policy(obs):
    return policy_fn(obs[None,:])
print('loading and building expert policy')
policy_fn = load_policy.load_policy('/Users/torbensell/Dropbox (Cambridge University)/UNI/CAM/PhD/Programme/BIRL_NN_pCN/homework/hw1/experts/HalfCheetah-v2.pkl')
print('loaded and built')
    
def discrete_action(action_vector):
    if np.random.uniform()<0.01:
        ''' With 1% probability, return a random action '''
        rand = np.random.randint(0,N_a)
        return rand,actions[rand]
    else:
        distances = np.zeros(N_a)
        for i in range(N_a):
            distances[i] = np.linalg.norm(action_vector-actions[i])
        action = np.argmin(distances) # index of action
        return action,actions[action]
    
def discrete_action_no_noise(action_vector):
    distances = np.zeros(N_a)
    for i in range(N_a):
        distances[i] = np.linalg.norm(action_vector-actions[i])
    action = np.argmin(distances) # index of action
    return action,actions[action]

loading and building expert policy
obs (1, 17) (1, 17)
loaded and built


In [3]:
"""

Create data and discretise action space

"""

N_a = 8
N_data = 500

with tf.Session():
    tf_util.initialize()   
    env = gym.make('HalfCheetah-v2')
    env = env.unwrapped
    
    ''' Set seed for reproducability '''
    env.seed(42)
    np.random.seed(42)
    
    obs = env.reset()

    data = np.zeros((N_data,dim+1)) # will store data here to be used in MCMC
    data_int = 0
    acts = np.zeros((N_a,6)) # will store N_a discretised actions here
    a_int = 0

    ''' discretise actions at every step, store the first 8. That's our discrete action space '''
    for k in range(1000):
        action = policy(obs)
        obs = env.step(action)[0]
        if a_int<N_a:
            acts[a_int,:] = action 
            a_int+=1

    np.save('HC_actions.npy',acts)


    ''' load discretised action space '''
    actions = []
    actions_matrix = np.load('HC_actions.npy')
    for i in range(N_a):
        actions.append(actions_matrix[i,:])


    ''' create data by looking firstly at the first 4 states of 25 different cheetahs, then the first 50 of another 8 cheetahs '''           
    for i in range(25):
        obs = env.reset()
        for k in range(4):
            action = policy(obs)
            action_int,disc_action = discrete_action_no_noise(action)
            if data_int<N_data:
                data[data_int,:] = np.append(obs,action_int) 
                data_int+=1
            obs = env.step(disc_action)[0]
    
    for i in range(8):
        obs = env.reset()
        for k in range(50):
            action = policy(obs)
            action_int,disc_action = discrete_action_no_noise(action)
            if data_int<N_data:
                data[data_int,:] = np.append(obs,action_int) 
                data_int+=1
            obs = env.step(disc_action)[0]
                    
    np.save('HC_data.npy',data)
    
    env.close()

Instructions for updating:
Please use tf.global_variables instead.
Instructions for updating:
Use `tf.variables_initializer` instead.


In [4]:
"""

Create points x_test in which we want to quantify decision uncertainty

"""

''' load discretised action space '''
N_a = 8
actions = []
actions_matrix = np.load('HC_actions.npy')
for i in range(N_a):
    actions.append(actions_matrix[i,:])
    
    
with tf.Session():
    tf_util.initialize()   
    env = gym.make('HalfCheetah-v2')
    env = env.unwrapped

    ''' Set seed for reproducability '''
    env.seed(42)
    np.random.seed(42)

    obs = env.reset()
    x_test = np.zeros((N_a,dim))
    reward_test = np.zeros(N_a)
    x_test_sorted = np.zeros((N_a,dim))
    reward_test_sorted = np.zeros(N_a)
    
    for k in range(1000):
        action = policy(obs)
        obs = env.step(action)[0]
        
        ''' the 100th point is the test point '''
        if k==100:
            pos = obs[0:8]
            vel = obs[8:17]
            pos = np.insert(pos,0, env.sim.data.qpos[0])
            
            action = policy(obs)
            action_int,disc_action = discrete_action_no_noise(action)
            
            ''' see where the actions would take us '''
            for i in range(N_a):
                x_test[i,:],reward_test[i] = env.step(actions[i])[0:2]
                env.set_state(pos, vel)
            
            ''' x_test such that the first entry is the taken action '''
            if action_int!=0:
                x_test_sorted[0,:] = x_test[action_int,:]
                x_test_sorted[1:action_int+1,:] = x_test[0:action_int,:]
                x_test_sorted[action_int+1:,:] = x_test[action_int+1:,:]
                reward_test_sorted[0] = reward_test[action_int]
                reward_test_sorted[1:action_int+1] = reward_test[0:action_int]
                reward_test_sorted[action_int+1:] = reward_test[action_int+1:]
            else:
                x_test_sorted = x_test
                reward_test_sorted = reward_test
                
                
    np.save('HC_x_test.npy',x_test_sorted)
    np.save('HC_reward_test.npy',reward_test_sorted)
            
    env.close()
    
    
    
with tf.Session():
    tf_util.initialize()   
    env = gym.make('HalfCheetah-v2')
    env = env.unwrapped
        
    ''' Set seed for reproducability '''
    env.seed(420)
    obs = env.reset()

    x_test = np.zeros((N_a,dim,100))
    a_test = np.zeros(100)
    j=0

    for k in range(1000):
        action = policy(obs)
        obs = env.step(action)[0]
        
        ''' every 5th point is a test point '''
        if k%5==0 and j<100:
            pos = obs[0:8]
            vel = obs[8:17]
            pos = np.insert(pos,0, env.sim.data.qpos[0])
            
            action = policy(obs)
            action_int,disc_action = discrete_action_no_noise(action)
            
            ''' see where the actions would take us '''
            for i in range(N_a):
                x_test[i,:,j] = env.step(actions[i])[0]
                env.set_state(pos, vel)
                
            a_test[j] = action_int
            j+=1
                
    np.save('HC_x_test_100.npy',x_test)
    np.save('HC_a_test_100.npy',a_test)
    
            
    env.close()

In [5]:
# with tf.Session():
#     tf_util.initialize()   
#     env = gym.make('HalfCheetah-v2')
#     env = env.unwrapped
#     for i in range(10):
#         obs = env.reset()
#         for k in range(1000):
#             obs = env.step(policy(obs))[0]
#             env.render()
#     env.close()